In [1]:
import multicelltypist
from datetime import date
import hisepy
import numpy as np
import os
import pandas as pd
import scanpy as sc

Retrieve 10x Genomics Flex Probe gene list

In [2]:
cache_res = hisepy.download_from_project_store(
    store_name = 'rds', # Reference Data Sets Project Store
    file_name = 'AIFI-2024-03-11T02:09:16.856602896Z/Chromium_Human_Transcriptome_Probe_Set_v1.0_GRCh38-2020-A.probe_metadata.tsv', # File from 10x Genomics
)

In [3]:
probe_file = 'rds/Chromium_Human_Transcriptome_Probe_Set_v1.0_GRCh38-2020-A.probe_metadata.tsv'
flex_probes = pd.read_csv(probe_file, sep = '\t')

In [4]:
flex_genes = flex_probes['gene_name'].unique().tolist()

In [5]:
len(flex_genes)

18529

In [6]:
def read_adata_uuid(h5ad_uuid):
    h5ad_path = '/home/jupyter/cache/{u}'.format(u = h5ad_uuid)
    if not os.path.isdir(h5ad_path):
        hise_res = hisepy.reader.cache_files([h5ad_uuid])
    h5ad_filename = os.listdir(h5ad_path)[0]
    h5ad_file = '{p}/{f}'.format(p = h5ad_path, f = h5ad_filename)
    adata = sc.read_h5ad(h5ad_file)
    return adata

In [7]:
def resample_anndata_min_max(adata, label_column, max_cells=None, min_cells=None, random_state = 3030):
    """
    Resamples an AnnData object based on the cell labels, with the option to resample with 
    replacement for classes below a specified threshold.

    Parameters:
    ad (AnnData): The AnnData object to be resampled.
    label_column (str): The column in ad.obs where the labels are stored.
    max_cells (int, optional): The maximum number of cells to keep per label. If None, no upper limit is applied.
    min_cells (int, optional): The minimum number of cells below which resampling with replacement occurs. If None, no lower limit is applied.
    random_state (int, default = 3030): An integer used to set the state of the numpy.random.Generator
    
    Returns:
    AnnData: The resampled AnnData object.
    """
    
    labels = adata.obs[label_column].unique()

    subsets = []

    rng = np.random.default_rng(random_state)
        
    for label in labels:
        # Subset AnnData object for the current label
        subset = adata.obs[adata.obs[label_column] == label]
    
        # Resample with replacement if the number of cells is below min_cells and min_cells is defined
        if min_cells is not None and subset.shape[0] < min_cells:
            subset = subset.sample(min_cells, replace = True, random_state = rng)
        # Resample without replacement if the number of cells is greater than max_cells and max_cells is defined
        elif max_cells is not None and subset.shape[0] > max_cells:
            subset = subset.sample(max_cells, replace = False, random_state = rng)
    
        subsets.append(subset)

    # Concatenate all subsets
    resampled_obs = pd.concat(subsets)
    
    resampled_adata = adata[resampled_obs.index]
    resampled_adata.obs_names_make_unique()

    return resampled_adata

In [8]:
label_column = 'AIFI_L3'
max_cell_number = 20000

## Read clean, annotated dataset

In [9]:
h5ad_uuid = '6e8972a5-9463-4230-84b4-a20de055b9c3'

In [10]:
adata = read_adata_uuid(h5ad_uuid)

In [11]:
adata.shape

(1823666, 1261)

In [12]:
adata.obs[label_column].value_counts()

AIFI_L3
Core naive CD4 T cell      341521
Core CD14 monocyte         217576
CM CD4 T cell              161769
Core naive CD8 T cell      115126
GZMK- CD56dim NK cell      102908
                            ...  
ASDC                          522
GZMK+ memory CD4 Treg         467
Activated memory B cell       433
CLP cell                      373
BaEoMaP cell                   78
Name: count, Length: 72, dtype: int64

## Sample and prepare reference data

In [13]:
adata_subset = resample_anndata_min_max(
    adata, 
    label_column, 
    max_cells = max_cell_number,
    random_state = 3030
)

In [14]:
adata_subset.shape

(636082, 1261)

In [15]:
adata_subset.obs[label_column].value_counts()

AIFI_L3
CM CD4 T cell                       20000
CM CD8 T cell                       20000
KLRF1+ GZMB+ CD27- EM CD8 T cell    20000
KLRF1- GZMB+ CD27- EM CD8 T cell    20000
Core naive CD4 T cell               20000
                                    ...  
ASDC                                  522
GZMK+ memory CD4 Treg                 467
Activated memory B cell               433
CLP cell                              373
BaEoMaP cell                           78
Name: count, Length: 72, dtype: int64

In [16]:
adata_subset = adata_subset.raw.to_adata()
adata_subset.shape

(636082, 33538)

### Limit to genes available in 10x Flex Probes

In [17]:
keep_var = adata_subset.var.index.isin(flex_genes)
sum(keep_var)

18329

In [18]:
adata_subset = adata_subset[:,keep_var]
adata_subset.shape

(636082, 18329)

In [19]:
sc.pp.normalize_total(adata_subset, target_sum=1e4)
sc.pp.log1p(adata_subset)

## Generate Initial model

In [ ]:
model_fs = multicelltypist.train(
    adata_subset, 
    label_column, 
    n_jobs = 60, 
    max_iter = 10, 
    multi_class = 'ovr', 
    use_SGD = True
)

🍳 Preparing data before training
✂️ 1096 non-expressed genes are filtered out
🔬 Input data has 636082 cells and 17233 genes
⚖️ Scaling input data


## Identify top features used for the model

Detected genes:

In [ ]:
df = adata_subset.X.toarray()
flag = df.sum(axis = 0) == 0
gene = adata_subset.var_names[ ~flag]

Features with high absolute classifier coefficients for each cell type class

`np.argpartition` will take the coefficient scores for each class, and retrieve the positions of the highest absolute coefficient scores to the end of an array of positions. We then select the `top_n` positions from the end of our array of positions, which allow us to retrieve genes with the highest absolute coefficients for each class.

We can then combine these to get a unique list of genes that are important for our model.

In [ ]:
top_n = 200

gene_index = np.argpartition(
    np.abs(model_fs.classifier.coef_),
    -top_n,
    axis = 1
)
gene_index = gene_index[:, -top_n:]
gene_index = np.unique(gene_index)

print('Number of genes selected: {n}'.format(n = len(gene_index)))

In [ ]:
selected_genes = gene[gene_index.tolist()]
selected_df = pd.DataFrame({'gene': selected_genes})

In [ ]:
selected_df.head()

## Generate full model using selected features

In [ ]:
adata = adata.raw.to_adata()

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [ ]:
adata = adata[:, adata.var_names.isin(selected_genes)]

In [ ]:
adata.shape

In [ ]:
model_fs = multicelltypist.train(
    adata, 
    label_column, 
    n_jobs = 60,
    max_iter = 100,
    multi_class = 'multinomial',
    check_expression = False
)

## Write outputs for storage

In [ ]:
out_dir = 'output'
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

In [ ]:
out_genes = 'output/ref_pbmc_clean_celltypist_top{n}_flex-features_{l}_{d}.csv'.format(
    n = top_n,
    l = label_column,
    d = date.today()
)

selected_df.to_csv(out_genes)

In [ ]:
out_model = 'output/ref_pbmc_clean_celltypist_model_flex-features_{l}_{d}.pkl'.format(
    l = label_column,
    d = date.today()
)

model_fs.write(out_model)

## Upload model to HISE

Finally, we'll use `hisepy.upload.upload_files()` to send a copy of our output to HISE to use for downstream analysis steps.

In [ ]:
study_space_uuid = '64097865-486d-43b3-8f94-74994e0a72e0'
title = 'PBMC Reference {l} CellTypist Model Flex Features {d}'.format(
    l = label_column,
    d = date.today()
)

In [ ]:
in_files = [h5ad_uuid]

In [ ]:
in_files

In [ ]:
out_files = [out_genes, out_model]

In [ ]:
out_files

In [ ]:
hisepy.upload.upload_files(
    files = out_files,
    study_space_id = study_space_uuid,
    title = title,
    input_file_ids = in_files
)

In [ ]:
import session_info
session_info.show()